In [72]:
from qiskit import QuantumCircuit

qc = QuantumCircuit(3)
qc.cx(0, 1)
qc.cx(1, 2)
qc.cx(0, 2)
qc.draw()

q_0: ──■─────────■──
     ┌─┴─┐       │  
q_1: ┤ X ├──■────┼──
     └───┘┌─┴─┐┌─┴─┐
q_2: ─────┤ X ├┤ X ├
          └───┘└───┘

In [73]:
# pass manager is given dag, so use this as input to our mock functions
from qiskit.converters import circuit_to_dag

dag = circuit_to_dag(qc)

In [69]:
# looking up index of a qubit in the circuit
for node in dag.topological_op_nodes():
    print([QuantumCircuit.find_bit(qc, arg).index for arg in node.qargs])

[0, 1]
[1, 2]
[0, 2]


In [70]:
# basic cost function
from qiskit.converters import dag_to_circuit


def get_cost(dag, layout):
    qc = dag_to_circuit(dag)
    cost = 0
    for node in dag.topological_op_nodes():
        bit_indices = [QuantumCircuit.find_bit(qc, arg).index for arg in node.qargs]
        # convert to physical qubit indices
        physical_indices = [layout[bit_index] for bit_index in bit_indices]
        # get the topological distance
        distance = abs(physical_indices[0] - physical_indices[1]) - 1
        cost += distance
    return cost


layout = {0: 0, 1: 1, 2: 2}
get_cost(dag, layout)

1

In [74]:
# cost function that updates layout on marked nodes
# mark the first op node in the dag with a new attribute
dag.op_nodes()[1].name = "cx_m"

In [75]:
# cost function that updates layout on marked nodes
from qiskit.converters import dag_to_circuit


def get_updated_cost(dag, layout):
    qc = dag_to_circuit(dag)
    cost = 0
    for node in dag.topological_op_nodes():
        bit_indices = [QuantumCircuit.find_bit(qc, arg).index for arg in node.qargs]
        # convert to physical qubit indices
        physical_indices = [layout[bit_index] for bit_index in bit_indices]
        # get the topological distance
        distance = abs(physical_indices[0] - physical_indices[1]) - 1
        cost += distance

        # update layout if marked node
        if node.name == "cx_m":
            layout[bit_indices[0]], layout[bit_indices[1]] = (
                layout[bit_indices[1]],
                layout[bit_indices[0]],
            )

    return cost


layout = {0: 0, 1: 1, 2: 2}
get_updated_cost(dag, layout)

0

In [76]:
for node in dag.two_qubit_ops():
    print(node.name)

cx
cx_m
cx
